In [16]:
import pandas as pd
import psycopg2
from datetime import datetime
import sys
sys.path.append('H:/uth_helpers/')
from db_utils import get_dsn

Official Counts

In [17]:
optz_counts = pd.read_csv('Z:/Users/iperez/optum_qa/transformed_record_counts/agg_counts/optum_zip_table_counts.csv')
optd_counts = pd.read_csv('Z:/Users/iperez/optum_qa/transformed_record_counts/agg_counts/optum_dod_table_counts.csv')

In [18]:
optz_counts.head()

,year,table,row_count,pat_count,start_date,end_date,complete_year
0,2018.0,confinement,2226950,1647222.0,2018-01-01 00:00:00,2018-12-31 00:00:00,1.0
1,2019.0,confinement,2296608,1684502.0,2019-01-01 00:00:00,2019-12-31 00:00:00,1.0
2,2020.0,confinement,2066527,1553634.0,2020-01-01 00:00:00,2020-12-31 00:00:00,1.0
3,2021.0,confinement,2238430,1671608.0,2021-01-01 00:00:00,2021-12-31 00:00:00,1.0
4,2022.0,confinement,1106741,830247.0,2022-01-01 00:00:00,2022-06-30 00:00:00,0.0


In [19]:
connection = psycopg2.connect(get_dsn())

greenplum optum counts, done already through **claimcounts_optz.sql**

In [20]:
gp_optz_counts = pd.read_sql('select * from dev.ip_optz_count;', con=connection)
gp_optd_counts = pd.read_sql('select * from dev.ip_optd_count;', con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [21]:
gp_optd_counts.head()

,data_source,date_created,year,row_count,pat_count,clm_count,table_source
0,gp_optd,02152023,2014.0,435132834,12178332.0,169820320.0,medical
1,gp_optd,02152023,2009.0,167522579,3129565.0,7140550.0,lab_result
2,gp_optd,02152023,2010.0,3455252,1103143.0,1517081.0,procedure
3,gp_optd,02152023,2017.0,243210189,12880507.0,243207555.0,rx
4,gp_optd,02152023,2022.0,1106741,760327.0,1106382.0,confinement


In [22]:
connection.close()

greenplum and official raw counts

In [23]:
optz_gp_raw_comparison = pd.merge(gp_optz_counts, optz_counts, left_on= ['year', 'table_source'], right_on=['year', 'table'], how='inner')
optz_gp_raw_comparison['row_difference'] = optz_gp_raw_comparison['row_count_y'] - optz_gp_raw_comparison['row_count_x']
optz_gp_raw_comparison['pat_difference'] = optz_gp_raw_comparison['pat_count_y'] - optz_gp_raw_comparison['pat_count_x']
optz_gp_raw_comparison = optz_gp_raw_comparison.loc[:, ['year', 'table', 'row_count_x', 'row_count_y', 'row_difference', 'pat_count_x', 'pat_count_y', 'pat_difference', 'complete_year']]
optz_gp_raw_comparison = optz_gp_raw_comparison.rename(columns={'row_count_x': 'greenplum_row_count', 'row_count_y': 'official_row_count', 'pat_count_x': 'greenplum_pat_count', 'pat_count_y': 'official_pat_count'})
optz_gp_raw_comparison = optz_gp_raw_comparison.sort_values(['year', 'table'])
optz_gp_raw_comparison.to_csv(f'optz_raw_gp_counts_{datetime.now().month}{datetime.now().day}{datetime.now().year}.csv', index=False)
optz_gp_raw_comparison 

,year,table,greenplum_row_count,official_row_count,row_difference,greenplum_pat_count,official_pat_count,pat_difference,complete_year
49,2007.0,confinement,1246509,1246509,0,947751.0,1063294.0,115543.0,1.0
98,2007.0,diagnostic,299413638,299413638,0,12402422.0,28876896.0,16474474.0,1.0
74,2007.0,lab_result,70145862,70145862,0,2169718.0,2964982.0,795264.0,1.0
77,2007.0,medical,329179801,329179801,0,12403136.0,28878111.0,16474975.0,1.0
40,2007.0,procedure,3701508,3701508,0,1290586.0,1469059.0,178473.0,1.0
...,...,...,...,...,...,...,...,...,...
3,NaN,lu_procedure,340926,341802,876,NaN,NaN,NaN,NaN
1,NaN,mbr_co_enroll,92445133,92445133,0,75815144.0,NaN,NaN,NaN
70,NaN,mbr_enroll,157260336,157260336,0,75815144.0,NaN,NaN,NaN
78,NaN,provider,10456400,10456400,0,NaN,NaN,NaN,NaN


In [24]:
optz_gp_raw_comparison[optz_gp_raw_comparison['row_difference'] != 0].sort_values('row_difference')

,year,table,greenplum_row_count,official_row_count,row_difference,greenplum_pat_count,official_pat_count,pat_difference,complete_year
3,NaN,lu_procedure,340926,341802,876,NaN,NaN,NaN,NaN
0,NaN,lu_diagnosis,332941,335394,2453,NaN,NaN,NaN,NaN
23,NaN,lu_ndc,495308,497927,2619,NaN,NaN,NaN,NaN


In [25]:
optd_gp_raw_comparison = pd.merge(gp_optd_counts, optd_counts, left_on= ['year', 'table_source'], right_on=['year', 'table'], how='inner')
optd_gp_raw_comparison['row_difference'] = optd_gp_raw_comparison['row_count_y'] - optd_gp_raw_comparison['row_count_x']
optd_gp_raw_comparison = optd_gp_raw_comparison.loc[:, ['year', 'table', 'row_count_x', 'row_count_y', 'row_difference', 'complete_year']]
optd_gp_raw_comparison = optd_gp_raw_comparison.rename(columns={'row_count_x': 'greenplum_row_count', 'row_count_y': 'official_row_count'})
optd_gp_raw_comparison = optd_gp_raw_comparison.sort_values(['year', 'table'])
optd_gp_raw_comparison.to_csv(f'optd_raw_gp_counts_{datetime.now().month}{datetime.now().day}{datetime.now().year}.csv', index=False)
optd_gp_raw_comparison                                                           

,year,table,greenplum_row_count,official_row_count,row_difference,complete_year
88,2007.0,confinement,1246509,1246509,0,1.0
22,2007.0,diagnostic,299413638,299413638,0,1.0
72,2007.0,lab_result,70145862,70145862,0,1.0
58,2007.0,medical,329179801,329179801,0,1.0
15,2007.0,procedure,3701508,3701508,0,1.0
...,...,...,...,...,...,...
14,NaN,mbr_co_enroll_r,92445133,92445133,0,NaN
48,NaN,mbr_enroll_r,143605773,143605773,0,NaN
74,NaN,mbrwdeath,4223226,4223226,0,NaN
87,NaN,provider,10456400,10456400,0,NaN


In [26]:
optd_gp_raw_comparison.sort_values(['year', 'table'])

,year,table,greenplum_row_count,official_row_count,row_difference,complete_year
88,2007.0,confinement,1246509,1246509,0,1.0
22,2007.0,diagnostic,299413638,299413638,0,1.0
72,2007.0,lab_result,70145862,70145862,0,1.0
58,2007.0,medical,329179801,329179801,0,1.0
15,2007.0,procedure,3701508,3701508,0,1.0
...,...,...,...,...,...,...
14,NaN,mbr_co_enroll_r,92445133,92445133,0,NaN
48,NaN,mbr_enroll_r,143605773,143605773,0,NaN
74,NaN,mbrwdeath,4223226,4223226,0,NaN
87,NaN,provider,10456400,10456400,0,NaN


In [27]:
optd_gp_raw_comparison[optd_gp_raw_comparison['row_difference'] != 0].sort_values('row_difference')

,year,table,greenplum_row_count,official_row_count,row_difference,complete_year
45,2021.0,rx,258319967,197405069,-60914898,0.0


Adding SQL Server Optum ZIP Counts

In [28]:
optz_counts = pd.read_csv('db_counts/local_optz_counts_021523.csv')
optz_counts = optz_counts.rename(columns={'table_source':'table'})
optz_counts.head()

,data_source,year,pat_count,clm_count,table,row_count,date_generated
0,local_optz,2007,12403136,140520668,medical,329179801.0,21523
1,local_optz,2007,12402422,140511176,diagnosis,299413638.0,21523
2,local_optz,2007,1290586,1784404,procedure,3701508.0,21523
3,local_optz,2007,947751,1246503,confinement,1246509.0,21523
4,local_optz,2007,11289925,155887015,rx,157976653.0,21523


In [29]:
optz_gp_raw_comparison.columns

Index(['year', 'table', 'greenplum_row_count', 'official_row_count',
       'row_difference', 'greenplum_pat_count', 'official_pat_count',
       'pat_difference', 'complete_year'],
      dtype='object')

In [31]:
optz_row_comparisons = optz_gp_raw_comparison.merge(optz_counts[['year', 'table','row_count', 'pat_count']], 
                                                    how='inner')
optz_row_comparisons = optz_row_comparisons.rename(columns={'row_count':'spcd_row_count', 'row_difference':'gp_official_row_difference', 'pat_count':'spcd_pat_count', 'pat_difference':'gp_official_pat_difference'})
optz_row_comparisons['spcd_official_row_difference'] = optz_row_comparisons['official_row_count'] - optz_row_comparisons['spcd_row_count']
optz_row_comparisons['spcd_official_pat_difference'] = optz_row_comparisons['official_pat_count'] - optz_row_comparisons['spcd_pat_count']

In [32]:
optz_row_comparisons

,year,table,greenplum_row_count,official_row_count,gp_official_row_difference,greenplum_pat_count,official_pat_count,gp_official_pat_difference,complete_year,spcd_row_count,spcd_pat_count,spcd_official_row_difference,spcd_official_pat_difference
0,2007.0,confinement,1246509,1246509,0,947751.0,1063294.0,115543.0,1.0,1246509.0,947751,0.0,115543.0
1,2007.0,lab_result,70145862,70145862,0,2169718.0,2964982.0,795264.0,1.0,70145862.0,2169718,0.0,795264.0
2,2007.0,medical,329179801,329179801,0,12403136.0,28878111.0,16474975.0,1.0,329179801.0,12403136,0.0,16474975.0
3,2007.0,procedure,3701508,3701508,0,1290586.0,1469059.0,178473.0,1.0,3701508.0,1290586,0.0,178473.0
4,2007.0,rx,157895113,157895113,0,11289235.0,28872992.0,17583757.0,1.0,157976653.0,11289925,-81540.0,17583067.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2021.0,confinement,2238430,2238430,0,1394600.0,1671608.0,277008.0,1.0,2231158.0,1392382,7272.0,279226.0
71,2021.0,lab_result,645502819,645502819,0,8959718.0,16794833.0,7835115.0,1.0,603344910.0,8915999,42157909.0,7878834.0
72,2021.0,medical,788904805,788904805,0,16438784.0,44875616.0,28436832.0,1.0,784920535.0,16425119,3984270.0,28450497.0
73,2021.0,procedure,4383112,4383112,0,861276.0,966162.0,104886.0,1.0,4332192.0,858409,50920.0,107753.0


In [ ]:
optz_row_comparisons[optz_row_comparisons['gp_official_difference'] != 0].sort_values(['gp_official_difference'])

,year,table,greenplum_row_count,official_row_count,gp_official_difference,complete_year,spcd_row_count,spcd_official_difference


In [ ]:
optz_row_comparisons[optz_row_comparisons['spcd_official_difference'] != 0].sort_values(['spcd_official_difference'])

,year,table,greenplum_row_count,official_row_count,gp_official_difference,complete_year,spcd_row_count,spcd_official_difference
66,2020.0,lab_result,539969676,539969676,0,1.0,665589859.0,-125620183.0
59,2018.0,rx,238405734,238405734,0,1.0,244050472.0,-5644738.0
54,2017.0,rx,237662101,237662101,0,1.0,243210189.0,-5548088.0
49,2016.0,rx,216393372,216393372,0,1.0,220205576.0,-3812204.0
44,2015.0,rx,181764040,181764040,0,1.0,185392931.0,-3628891.0
29,2012.0,rx,176009405,176009405,0,1.0,179614435.0,-3605030.0
39,2014.0,rx,174168594,174168594,0,1.0,177644564.0,-3475970.0
34,2013.0,rx,184328487,184328487,0,1.0,185113018.0,-784531.0
24,2011.0,rx,170376326,170376326,0,1.0,171044225.0,-667899.0
19,2010.0,rx,163099773,163099773,0,1.0,163298527.0,-198754.0
